## Config

In [5]:
import re
import os
import glob
import random
import cv2
import numpy as np
import json
import pandas as pd
import shutil
import os
import zipfile
from kaggle_secrets import UserSecretsClient
from huggingface_hub import upload_file, HfApi
HF_APIKEY = UserSecretsClient().get_secret("HF-APIKEY")
api = HfApi(token=HF_APIKEY)

SRC_DIR = '/kaggle/input/pollen20ldet/'
IMAGE_DIR = '/kaggle/input/pollen20ldet/images'
DEST_DIR = 'data/dataset'
PHASES = ["train", "valid", "test"]
TRAIN_DIRECTORY = DEST_DIR + '/train'
VAL_DIRECTORY = DEST_DIR + '/valid'
TEST_DIRECTORY = DEST_DIR + '/test'

## Preprocess and Split

- Split into train/valid/test sets
- Convert from Pascal VOC (csv) to COCO JSON format 
- Zip each phase
  - train/*.jpg (images)
  - train/train.json (annotations)
  - valid/*
  - test/*

In [ ]:
shutil.copyfile(SRC_DIR, DEST_DIR+'/full')

In [ ]:
def convert_voc_to_yolo(voc_bbox, img_width, img_height):
    x_min, y_min, x_max, y_max = voc_bbox
    x_center = (x_min + x_max) / 2 / img_width
    y_center = (y_min + y_max) / 2 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    return x_center, y_center, width, height

class DatasetPreprocessor:
    def __init__(self, src_dir, image_dir, dest_dir, phases=["train","valid","test"]):
        self.src_dir = src_dir
        self.image_dir = image_dir
        self.dest_dir = dest_dir
        self.phases = phases
        self.annotation_file_name = ".json"
        
    def zip_each_phase(self):
        for phase in self.phases:
            phase_dir = os.path.join(self.dest_dir, phase)
            zip_file = os.path.join(self.dest_dir, f"{phase}.zip")
            with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
                for root, _, files in os.walk(phase_dir):
                    for file in files:
                        zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), phase_dir))

    def split_dataset(self, dataset_df):
        # Define the percentages for train, valid, and test splits
        train_percentage = 0.75
        valid_percentage = 0.15
        test_percentage = 0.10

        full_train_images = []
        full_valid_images = []
        full_test_images = []

        for class_name, sub_df in dataset_df.groupby(['class_name']):
            num_images = sub_df.shape[0]
            # Ensure at least one image in each phase
            num_train = max(1, int(train_percentage * num_images))
            num_valid = max(1, int(valid_percentage * num_images))
            num_test = max(1, int(test_percentage * num_images))

            # Take random sample for test and valid sets. Train will be leftover

            test_images = list(sub_df.sample(n=num_test, random_state=1).image_name.values)
            valid_images = list(sub_df[~sub_df.image_name.isin(test_images)].sample(n=num_valid, random_state=1).image_name.values)
            train_images = list(sub_df[~sub_df.image_name.isin(valid_images + test_images)].image_name.values)
            # Create directories for train, valid, and test splits
            full_train_images.extend(train_images)
            full_valid_images.extend(valid_images)
            full_test_images.extend(test_images)
            
        if len(self.phases) == 2:
            full_train_images = full_train_images + full_test_images
            images_phases = zip(self.phases, [full_train_images, full_valid_images])
        else:
            images_phases = zip(self.phases, [full_train_images, full_valid_images, full_test_images])

        for phase, image_names in images_phases:
            os.makedirs(os.path.join(self.dest_dir, phase), exist_ok=True)
            for image_name in image_names:
                src_path = os.path.join(self.image_dir, image_name)
                dest_path = os.path.join(self.dest_dir, phase, image_name)
                shutil.copyfile(src_path, dest_path)

    def annotate_dataset(self, categories, bboxes_df, inverse_class_dict):
        for phase in self.phases:
            root_path = os.path.join(self.dest_dir, phase)
            json_file = os.path.join(self.dest_dir, f"{phase}/_annotations.json")
            res_file = {
                "categories": categories,
                "images": [],
                "annotations": []
            }
            annot_count = 0
            image_id = 0
            processed = 0

            # Obtain image_names
            file_list = glob.glob(os.path.join(root_path, "*.jpg"))

            for file in file_list:
                image_path = file
                file_name = file.replace(root_path, "").replace("\\", '').replace('/', '')
                img = cv2.imread(image_path)
                img_h, img_w, channels = img.shape
                img_elem = {
                    "file_name": file_name,
                    "height": img_h,
                    "width": img_w,
                    "id": image_id
                }
                res_file["images"].append(img_elem)
                annotations = bboxes_df[bboxes_df.image_name == file_name]
                for idx, row in annotations.iterrows():
                    key = row['image_name']
                    voc_bbox = row['x1'], row['y1'], row['x2'], row['y2']
                    x1, y1, x2, y2 = convert_voc_to_yolo(voc_bbox, img_w, img_h)
                    coords = inverse_class_dict.get(row['class_name']), x1, y1, x2, y2

                    # YOLO to COCO JSON
                    x_center = (float(coords[1]) * (img_w))
                    y_center = (float(coords[2]) * (img_h))
                    width = (float(coords[3]) * img_w)
                    height = (float(coords[4]) * img_h)
                    category_id = int(coords[0])  # label

                    mid_x = int(x_center - width / 2)
                    mid_y = int(y_center - height / 2)
                    width = int(width)
                    height = int(height)

                    area = width * height
                    poly = [[mid_x, mid_y],
                            [width, height],
                            [width, height],
                            [mid_x, mid_y]]

                    annot_elem = {
                        "id": annot_count,
                        "bbox": [
                            float(mid_x),
                            float(mid_y),
                            float(width),
                            float(height)
                        ],
                        "segmentation": list([poly]),
                        "image_id": image_id,
                        "ignore": 0,
                        "category_id": category_id,
                        "iscrowd": 0,
                        "area": float(area)
                    }
                    res_file["annotations"].append(annot_elem)
                    annot_count += 1
                image_id += 1

                processed += 1
            with open(json_file, "w") as f:
                json_str = json.dumps(res_file)
                f.write(json_str)

            print("Processed {} {} images...".format(processed, phase))
        print("Done.")

    def process_dataset(self, bboxes_file, class_map_file, dataloader_batchsize=2):
        print('Processing Dataset...')
        bboxes_df = pd.read_csv(bboxes_file, header=None, names=['image_name', 'x1', 'y1', 'x2', 'y2', 'class_name'])
        
        labels_df = pd.read_csv(class_map_file, header=None, names=['name', 'id'])
        class_dict = dict(zip(list(map(int, labels_df.id.values)), list(labels_df.name.values)))
        inverse_class_dict = dict(zip(list(labels_df.name.values), list(map(int,labels_df.id.values))))
        categories = [{"supercategory": "none", "name": val, "id": int(key)} for key, val in class_dict.items()]

        dataset_df = bboxes_df[['image_name', 'class_name']].drop_duplicates(['image_name'])
        dataset_df.class_name.value_counts()
        print(f'  -- Images: {dataset_df.shape[0]}')
        print(f'  -- Annotations: {bboxes_df.shape[0]}')
        print(f'  -- Classes: {labels_df.shape[0]}')
        print('')
        print('Splitting Dataset...')
        self.split_dataset(dataset_df)
        print('Processing Dataset...')
        self.annotate_dataset(categories, bboxes_df, inverse_class_dict)
        print('Zipping each phase...')
        self.zip_each_phase()
        
        ### ZIP EACH self.phases (train, valid, test)
        return class_dict, inverse_class_dict
        
        
dataset_preprocessor = DatasetPreprocessor(
    src_dir=SRC_DIR,
    image_dir=IMAGE_DIR,
    dest_dir=DEST_DIR,
    phases=PHASES
)
id2label, label2id = dataset_preprocessor.process_dataset(
    bboxes_file=f"{SRC_DIR}bboxes.csv",
    class_map_file=f"{SRC_DIR}class_map.csv"
)

## Save Dataset Loader

In [ ]:
%%writefile /kaggle/working/pollen_detection_loader.py

import collections
import json
import os

import datasets


_HOMEPAGE = "https://www.kaggle.com/datasets/nataliakhanzhina/pollen20ldet"
_LICENSE = "CC BY 4.0"
_CITATION = """\
@misc{ pollen20ldet,
    title = { Combating data incompetence in pollen images detection and classification for pollinosis prevention },
    type = { Open Source Dataset },
    author = { Khanzhina, Natalia and Filchenkov, Andrey and Minaeva, Natalia and Novoselova, Larisa and Petukhov, Maxim and Kharisova, Irina and Pinaeva, Julia and Zamorin, Georgiy and Putin, Evgeny and Zamyatina, Elena and others},
    howpublished = { \\url{ https://www.kaggle.com/datasets/nataliakhanzhina/pollen20ldet } },
    url = { https://www.kaggle.com/datasets/nataliakhanzhina/pollen20ldet },
    journal = { Computers in biology and medicine },
    volume={140},
    pages={105064},
    publisher = { Elsevier },
    year = { 2022 },
}
"""

### I want to look at multiple ways to load categories. Pollen has multiple classification levels and is usually classified via latin version of the term

_CATEGORIES = [
    'buckwheat',
    'clover',
    'angelica',
    'angelica_garden',
    'willow',
    'hill_mustard',
    'linden',
    'meadow_pink',
    'alder',
    'birch',
    'fireweed',
    'nettle',
    'pigweed',
    'plantain',
    'sorrel',
    'grass',
    'pine',
    'maple',
    'hazel',
    'mugwort'
]

_ANNOTATION_FILENAME = "_annotations.json"


class POLLENDETECTIONConfig(datasets.BuilderConfig):
    """Builder Config for pollen-detection"""

    def __init__(self, data_urls, **kwargs):
        """
        BuilderConfig for pollen-detection.
        Args:
          data_urls: `dict`, name to url to download the zip file from.
          **kwargs: keyword arguments forwarded to super.
        """
        super(POLLENDETECTIONConfig, self).__init__(version=datasets.Version("1.0.0"), **kwargs)
        self.data_urls = data_urls


class POLLENDETECTION(datasets.GeneratorBasedBuilder):
    """pollen-detection object detection dataset"""

    VERSION = datasets.Version("1.0.0")
    BUILDER_CONFIGS = [
        POLLENDETECTIONConfig(
            name="full",
            description="Full version of pollen-detection dataset.",
            data_urls={
                "train": "https://huggingface.co/datasets/Charliesgt/Pollen20LDet/resolve/main/data/train.zip",
                "valid": "https://huggingface.co/datasets/Charliesgt/Pollen20LDet/resolve/main/data/valid.zip",
                "test": "https://huggingface.co/datasets/Charliesgt/Pollen20LDet/resolve/main/data/test.zip",
            },
        ),
    ]

    def _info(self):
        features = datasets.Features(
            {
                "image_id": datasets.Value("int64"),
                "image": datasets.Image(),
                "width": datasets.Value("int32"),
                "height": datasets.Value("int32"),
                "objects": datasets.Sequence(
                    {
                        "id": datasets.Value("int64"),
                        "area": datasets.Value("int64"),
                        "bbox": datasets.Sequence(datasets.Value("float32"), length=4),
                        "category": datasets.ClassLabel(names=_CATEGORIES),
                    }
                ),
            }
        )
        return datasets.DatasetInfo(
            features=features,
            homepage=_HOMEPAGE,
            citation=_CITATION,
            license=_LICENSE,
        )

    def _split_generators(self, dl_manager):
        data_files = dl_manager.download_and_extract(self.config.data_urls)
        return [
            datasets.SplitGenerator(
                name=datasets.Split.TRAIN,
                gen_kwargs={
                    "folder_dir": data_files["train"],
                },
            ),
            datasets.SplitGenerator(
                name=datasets.Split.VALIDATION,
                gen_kwargs={
                    "folder_dir": data_files["valid"],
                },
            ),
            datasets.SplitGenerator(
                name=datasets.Split.TEST,
                gen_kwargs={
                    "folder_dir": data_files["test"],
                },
            ),
]

    def _generate_examples(self, folder_dir):
        def process_annot(annot, category_id_to_category):
            return {
                "id": annot["id"],
                "area": annot["area"],
                "bbox": annot["bbox"],
                "category": category_id_to_category[annot["category_id"]],
            }

        image_id_to_image = {}
        idx = 0

        annotation_filepath = os.path.join(folder_dir, _ANNOTATION_FILENAME)
        with open(annotation_filepath, "r") as f:
            annotations = json.load(f)
        category_id_to_category = {category["id"]: category["name"] for category in annotations["categories"]}
        image_id_to_annotations = collections.defaultdict(list)
        for annot in annotations["annotations"]:
            image_id_to_annotations[annot["image_id"]].append(annot)
        filename_to_image = {image["file_name"]: image for image in annotations["images"]}

        for filename in os.listdir(folder_dir):
            filepath = os.path.join(folder_dir, filename)
            if filename in filename_to_image:
                image = filename_to_image[filename]
                objects = [
                    process_annot(annot, category_id_to_category) for annot in image_id_to_annotations[image["id"]]
                ]
                with open(filepath, "rb") as f:
                    image_bytes = f.read()
                yield idx, {
                    "image_id": image["id"],
                    "image": {"path": filepath, "bytes": image_bytes},
                    "width": image["width"],
                    "height": image["height"],
                    "objects": objects,
                }
                idx += 1



In [ ]:
# Define your dataset name

dataset_name = "Pollen20LDet"

repo_id = f"Charliesgt/{dataset_name}"
repo_type='dataset'

# Define paths to your files
train_zip_path = f"{DEST_DIR}/train.zip"
valid_zip_path = f"{DEST_DIR}/valid.zip"
test_zip_path = f"{DEST_DIR}/test.zip"
loader_script_path = "/kaggle/working/pollen_detection_loader.py"

# Create directory structure
data_dir = os.path.join(dataset_name, "data")
os.makedirs(data_dir, exist_ok=True)

# Move zip files to data folder
shutil.copy(train_zip_path, os.path.join(data_dir, "train.zip"))
shutil.copy(valid_zip_path, os.path.join(data_dir, "valid.zip"))
shutil.copy(test_zip_path, os.path.join(data_dir, "test.zip"))

# Upload dataset files to Hugging Face Dataset Hub
api.upload_folder(folder_path=data_dir,path_in_repo = 'data', repo_id=repo_id, repo_type=repo_type)

# Upload dataset loader script
api.upload_file(path_or_fileobj=loader_script_path,path_in_repo='Pollen20LDet.py',repo_id=repo_id, repo_type=repo_type, commit_message="Upload dataset loader script")

In [ ]:
from datasets import load_dataset, concatenate_datasets, DatasetDict
dataset = load_dataset("Charliesgt/Pollen20LDet", trust_remote_code=True)




In [9]:
import requests
import json
import glob
def load_dataset_local(repo_id ,splits, dest_dir):
    if os.path.exists(f'/kaggle/working/{dest_dir}'):
        shutil.rmtree(f'/kaggle/working/{dest_dir}')
    
    split_path = f"https://huggingface.co/datasets/{repo_id}/resolve/main/data/"
    os.makedirs(dest_dir, exist_ok=True)
    for split in splits:
        # Download the zip file
        response = requests.get(split_path + f"{split}.zip")
        os.makedirs(os.path.join(dest_dir, split), exist_ok=True)
        
        # Save the zip file
        save_dir = os.path.join(dest_dir, split)
        zip_file_path = os.path.join(save_dir, f"{split}.zip")
        with open(zip_file_path, "wb") as f:
            f.write(response.content)
        # Unzip the downloaded file
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(save_dir)

        # Remove the zip file if you no longer need it
        os.remove(zip_file_path)

load_dataset_local('Charliesgt/Pollen20LDet',PHASES, DEST_DIR)

In [ ]:
class PollenDetection(torchvision.datasets.CocoDetection):
    def __init__(
        self, 
        image_directory_path: str, 
        image_processor, 
        train: bool = True
    ):
        annotation_file_path = image_directory_path + '.json'
        super(PollenDetection, self).__init__(image_directory_path, annotation_file_path)
        self.image_processor = image_processor

    def __getitem__(self, idx):
        images, annotations = super(PollenDetection, self).__getitem__(idx)        
        image_id = self.ids[idx]
        annotations = {'image_id': image_id, 'annotations': annotations}
        encoding = self.image_processor(images=images, annotations=annotations, return_tensors="pt")
        pixel_values = encoding["pixel_values"].squeeze()
        target = encoding["labels"][0]
        return pixel_values, target
    
    
    
        def collate_fn(batch):
            # DETR authors employ various image sizes during training, making it not possible
            # to directly batch together images. Hence they pad the images to the biggest
            # resolution in a given batch, and create a corresponding binary pixel_mask
            # which indicates which pixels are real/which are padding
            pixel_values = [item[0] for item in batch]
            encoding = image_processor.pad(pixel_values, return_tensors="pt")
            labels = [item[1] for item in batch]
            return {
                'pixel_values': encoding['pixel_values'],
                'pixel_mask': encoding['pixel_mask'],
                'labels': labels
            }

        train_dataset = PollenDetection(
            image_directory_path=TRAIN_DIRECTORY,
            image_processor=image_processor,
            train=True
        )
        val_dataset = PollenDetection(
            image_directory_path=VAL_DIRECTORY,
            image_processor=image_processor,
            train=False
        )
        train_dataloader = DataLoader(dataset=train_dataset, collate_fn=collate_fn, batch_size=dataloader_batchsize, shuffle=True, num_workers=4)
        val_dataloader = DataLoader(dataset=val_dataset, collate_fn=collate_fn, batch_size=dataloader_batchsize, num_workers=2)
        return train_dataloader, val_dataloader, class_dict, inverse_class_dict